In [1]:
import pandas as pd, json, requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


In [ ]:
import json
import pandas as pd

champion_data = {}

with open("../data/championFull copy.json", encoding="utf8") as f:
    data = json.load(f)
    
    for champ_name, champ_data in data['data'].items():
        stats = champ_data['stats']
        info = champ_data['info']
        
        # Merge stats and info dict into single dict
        merged = stats.copy()  
        merged.update(info)
        
        champion_data[champ_name] = merged

# Convert to DataFrame 
print(champion_data.items())

In [5]:
# Get all actors as an index for a dataframe
index = champion_data.keys()

# Get the genre-counts for each actor in the index
rows = [champion_data[k] for k in index]

# Create the data frame from these rows, with the actors as index
df = pd.DataFrame(rows, index=index)

# Fill NAs with zero, as NA means the actor has not starred in that genre
df = df.fillna(0)

df

,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,...,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,attack,defense,magic,difficulty
Aatrox,650,114,0,0.0,345,38,4.45,32,2.05,175,...,0,0,60,5.00,2.500,0.651,8,4,3,4
Ahri,590,96,418,25.0,330,21,4.70,30,1.30,550,...,0,0,53,3.00,2.200,0.668,3,4,8,5
Akali,570,119,200,0.0,345,23,4.70,37,2.05,125,...,0,0,62,3.30,3.200,0.625,5,3,8,7
Akshan,630,107,350,40.0,330,26,4.70,30,1.30,500,...,0,0,52,3.00,4.000,0.638,0,0,0,0
Alistar,685,120,350,40.0,330,47,4.70,32,2.05,125,...,0,0,62,3.75,2.125,0.625,6,9,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zeri,630,110,250,45.0,330,24,4.20,30,1.30,500,...,0,0,53,1.30,2.000,0.658,8,5,3,6
Ziggs,606,106,480,23.5,325,18,4.50,30,1.30,550,...,0,0,55,3.10,2.000,0.656,2,4,9,4
Zilean,574,96,452,50.0,335,24,5.00,30,1.30,550,...,0,0,52,3.00,2.130,0.658,2,5,8,6
Zoe,630,106,425,25.0,340,21,4.70,30,1.30,550,...,0,0,58,3.30,2.500,0.658,1,7,8,5


In [ ]:
k = 5

cluster_model = KMeans(n_clusters=k)
cluster_model.fit(df)

cluster_labels = cluster_model.predict(df)
champion_cluster_df = pd.DataFrame(cluster_labels, index=df.index, columns=["cluster"])
champion_cluster_df["cluster"].value_counts()
scores = []
for k in range(2, 32):
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(df)
    score = silhouette_score(df, labels)
    scores.append((k, score))
  
ks, scores = zip(*scores)
plt.plot(ks, scores)
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.title('Silhouette Score vs K')

k = 8
kmeans = KMeans(n_clusters=k).fit(df) 
for i in range(k):
    cluster_df = df[kmeans.labels_==i] 
    print(f'Cluster {i}:') 
    for j in range(5):
        print(cluster_df.sample(1).index[0])